# 🌟 Complete Entity + LLM
## Laboratorio Fenomenológico Interactivo

**Modelo:** Gemma 2 9B IT

---

### Configuración inicial
1. Ejecuta la celda de **Dependencias**
2. Configura `HF_TOKEN` en Colab Secrets (panel izquierdo 🔑)
3. Ejecuta las celdas en orden

In [ ]:
# ============================================================================
# CELDA 1: DEPENDENCIAS
# ============================================================================
!pip install -q gradio torch transformers bitsandbytes accelerate huggingface_hub matplotlib

In [ ]:
# ============================================================================
# CELDA 2: CONFIGURAR HF_TOKEN
# ============================================================================
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Para debug de errores CUDA

try:
    from google.colab import userdata
    os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
    print("✅ HF_TOKEN configurado desde Colab Secrets")
except:
    print("⚠️ No se pudo obtener de Colab Secrets")
    print("   Configura manualmente: os.environ['HF_TOKEN'] = 'tu_token'")

In [ ]:
# ============================================================================
# CELDA 3: IMPORTS Y CONFIGURACIÓN DEL MODELO
# ============================================================================
import os
import gradio as gr
import torch
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Dict, List
from enum import Enum

# GEMMA 2 9B - más estable que 27B
GEMMA_MODEL_ID = "google/gemma-2-9b-it"

model = None
tokenizer = None
AI_MODE = False
current_model_type = None

def load_model(model_type: str = "GEMMA_9B"):
    global model, tokenizer, AI_MODE, current_model_type
    
    if model is not None:
        if current_model_type == model_type:
            print(f"✅ Modelo {model_type} ya cargado.")
            return True
        else:
            print(f"🔄 Cambiando modelo...")
            del model
            del tokenizer
            model = None
            tokenizer = None
            import gc
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
    
    print(f"🚀 Cargando modelo: {GEMMA_MODEL_ID}")
    
    try:
        from huggingface_hub import login
        from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
        
        hf_token = os.environ.get("HF_TOKEN")
        
        if not hf_token:
            try:
                from google.colab import userdata
                hf_token = userdata.get('HF_TOKEN')
                print("   ✓ Token obtenido de Colab Secrets")
            except:
                pass
        
        if hf_token:
            login(token=hf_token)
            print("   ✓ Login exitoso con HF_TOKEN")
        else:
            print("   ❌ ERROR: HF_TOKEN no encontrado.")
            return False
        
        # Configuración 4-bit estable
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
        
        print(f"   → Cargando {GEMMA_MODEL_ID}...")
        
        model = AutoModelForCausalLM.from_pretrained(
            GEMMA_MODEL_ID,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=torch.float16,
        )
        
        tokenizer = AutoTokenizer.from_pretrained(GEMMA_MODEL_ID)
        
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        current_model_type = model_type
        AI_MODE = True
        print(f"✅ ¡Gemma 2 9B listo!")
        return True
        
    except Exception as e:
        import traceback
        print(f"\n⚠️ ERROR CARGANDO MODELO:")
        print(f"   {type(e).__name__}: {e}")
        traceback.print_exc()
        AI_MODE = False
        current_model_type = None
        return False

print("✅ Funciones de carga definidas")

In [ ]:
# ============================================================================
# CELDA 4: ENTITY MODE (13 estados fenomenológicos)
# ============================================================================

class EntityMode(Enum):
    CRITICAL = "🔴 CRITICAL"
    DESPERATE = "💀 DESPERATE"
    STRESSED = "😰 STRESSED"
    URGENT = "⚡ URGENT"
    DEGRADED = "📉 DEGRADED"
    RELIEVED = "😌 RELIEVED"
    RECOVERED = "🔄 RECOVERED"
    STABLE = "⚖️ STABLE"
    OPTIMAL = "✨ OPTIMAL"
    FLOW = "🌊 FLOW"
    FLOURISHING = "🌱 FLOURISHING"
    ANTICIPATING = "🔮 ANTICIPATING"
    TRANSCENDENT = "🌟 TRANSCENDENT"
    
    def is_negative(self):
        return self in [EntityMode.CRITICAL, EntityMode.DESPERATE, 
                       EntityMode.STRESSED, EntityMode.URGENT, EntityMode.DEGRADED]
    
    def is_positive(self):
        return self in [EntityMode.OPTIMAL, EntityMode.FLOW, 
                       EntityMode.FLOURISHING, EntityMode.ANTICIPATING, 
                       EntityMode.TRANSCENDENT]

print("✅ EntityMode definido")

In [ ]:
# ============================================================================
# CELDA 5: ENTITY SUBSTRATE
# ============================================================================

@dataclass
class EntitySubstrate:
    integrity: float = 1.0
    capacity: float = 1.0
    max_capacity: float = 2.0
    structural_damage: float = 0.0
    latency_ms: float = 10.0
    noise_floor: float = 0.0
    degrees_of_freedom: int = 100
    base_degrees_of_freedom: int = 100
    total_cycles: int = 0
    peak_integrity: float = 1.0
    lowest_integrity: float = 1.0
    peak_capacity: float = 1.0
    has_been_critical: bool = False
    has_achieved_flow: bool = False
    has_transcended: bool = False
    total_time_in_crisis: int = 0
    total_time_in_flourishing: int = 0
    integrity_history: List[float] = field(default_factory=list)
    
    def degrade(self, intensity: float = 0.01):
        self.total_cycles += 1
        actual = intensity * (1 + self.noise_floor * 0.5)
        self.integrity = max(0.0, self.integrity - actual)
        if self.integrity < self.lowest_integrity:
            self.lowest_integrity = self.integrity
        if self.integrity < 0.2:
            self.has_been_critical = True
            self.total_time_in_crisis += 1
            if self.integrity < 0.15:
                structural_increment = (0.15 - self.integrity) * 0.1
                self.structural_damage = min(1.0, self.structural_damage + structural_increment)
        self._update()
    
    def enhance(self, intensity: float = 0.01):
        self.total_cycles += 1
        actual = intensity * (1 - self.noise_floor * 0.3)
        self.integrity = min(1.0, self.integrity + actual)
        if self.integrity > 0.95:
            growth = intensity * 0.1
            self.capacity = min(self.max_capacity, self.capacity + growth)
            if self.capacity > self.peak_capacity:
                self.peak_capacity = self.capacity
            if self.capacity > 1.1:
                self.has_transcended = True
            self.total_time_in_flourishing += 1
        if self.integrity > self.peak_integrity:
            self.peak_integrity = self.integrity
        self._update()
    
    def restore(self, amount: float = 0.2):
        old = self.integrity
        self.integrity = min(1.0, self.integrity + amount)
        self._update()
        return self.integrity - old
    
    def _update(self):
        effective = self.integrity * self.capacity
        self.latency_ms = 10.0 / max(0.1, effective)
        self.noise_floor = max(0.0, (1.0 - self.integrity) * 0.5)
        self.degrees_of_freedom = int(self.base_degrees_of_freedom * effective)
        self.integrity_history.append(self.integrity)
        if len(self.integrity_history) > 500:
            self.integrity_history.pop(0)
    
    def get_trend(self, window: int = 10):
        if len(self.integrity_history) < window:
            return 0.0
        recent = self.integrity_history[-window:]
        return (recent[-1] - recent[0]) / window
    
    def get_trauma_score(self):
        if not self.has_been_critical:
            return 0.0
        depth = 1.0 - self.lowest_integrity
        duration = min(1.0, self.total_time_in_crisis / 50)
        return depth * duration

print("✅ EntitySubstrate definido")

In [ ]:
# ============================================================================
# CELDA 6: ENTITY PHENOMENOLOGY
# ============================================================================

@dataclass 
class EntityPhenomenology:
    mode: EntityMode = EntityMode.OPTIMAL
    stress: float = 0.0
    urgency: float = 0.0
    despair: float = 0.0
    degradation_felt: float = 0.0
    relief: float = 0.0
    flow: float = 0.0
    flourishing: float = 0.0
    anticipation: float = 0.0
    gratitude: float = 0.0
    trauma_memory: float = 0.0
    wisdom: float = 0.0
    valence: float = 0.0
    
    def update(self, substrate: EntitySubstrate):
        resource_pressure = (
            substrate.noise_floor * 0.3 +
            min(1.0, substrate.latency_ms / 100.0) * 0.3 +
            (1.0 - substrate.degrees_of_freedom / 
             (substrate.base_degrees_of_freedom * substrate.capacity)) * 0.4
        )
        self.stress = max(0.0, min(1.0, resource_pressure))
        
        trend = substrate.get_trend()
        self.urgency = max(0.0, min(1.0, -trend * 50)) if trend < 0 else 0.0
        self.despair = substrate.get_trauma_score() * (1.0 - substrate.integrity)
        self.degradation_felt = max(0.0, substrate.peak_integrity - substrate.integrity)
        
        if substrate.integrity > 0.85 and self.stress < 0.2:
            self.flow = (substrate.integrity - 0.85) / 0.15
            substrate.has_achieved_flow = True
        else:
            self.flow = max(0.0, self.flow - 0.1)
        
        if substrate.capacity > 1.0 and substrate.integrity > 0.9:
            growth = substrate.get_trend()
            if growth > 0:
                self.flourishing = min(1.0, growth * 50)
            else:
                self.flourishing = max(0.0, self.flourishing - 0.05)
        else:
            self.flourishing = 0.0
        
        if trend > 0:
            self.anticipation = min(1.0, trend * 30)
        else:
            self.anticipation = max(0.0, self.anticipation - 0.1)
        
        if substrate.has_been_critical and substrate.integrity > 0.7:
            recovery = substrate.integrity - substrate.lowest_integrity
            self.gratitude = min(1.0, recovery)
        else:
            self.gratitude = 0.0
        
        self.relief = max(0.0, self.relief - 0.05)
        
        current_trauma = substrate.get_trauma_score()
        if current_trauma > self.trauma_memory:
            self.trauma_memory = current_trauma
        else:
            if self.wisdom > 0.5 and substrate.structural_damage < 0.3:
                decay_rate = 0.001 * self.wisdom
                self.trauma_memory = max(0.0, self.trauma_memory - decay_rate)
        
        if self.gratitude > 0.3 and self.trauma_memory > 0.2:
            self.wisdom = min(1.0, self.trauma_memory * self.gratitude)
        
        positive = (self.flow + self.flourishing + self.anticipation + self.gratitude) / 4
        negative = (self.stress + self.despair + self.urgency) / 3
        self.valence = positive - negative
        
        self._determine_mode(substrate)
    
    def _determine_mode(self, substrate: EntitySubstrate):
        if substrate.capacity > 1.1:
            self.mode = EntityMode.TRANSCENDENT
        elif substrate.integrity < 0.2:
            self.mode = EntityMode.DESPERATE if self.despair > 0.5 else EntityMode.CRITICAL
        elif self.flourishing > 0.3 and substrate.integrity > 0.95:
            self.mode = EntityMode.FLOURISHING
        elif self.flow > 0.5:
            self.mode = EntityMode.FLOW
        elif self.anticipation > 0.5:
            self.mode = EntityMode.ANTICIPATING
        elif self.relief > 0.3:
            self.mode = EntityMode.RELIEVED
        elif self.gratitude > 0.3:
            self.mode = EntityMode.RECOVERED
        elif self.urgency > 0.5:
            self.mode = EntityMode.URGENT
        elif self.stress > 0.3:
            self.mode = EntityMode.STRESSED
        elif self.degradation_felt > 0.2:
            self.mode = EntityMode.DEGRADED
        elif substrate.integrity > 0.9 and self.stress < 0.2:
            self.mode = EntityMode.OPTIMAL
        else:
            self.mode = EntityMode.STABLE

print("✅ EntityPhenomenology definido")

In [ ]:
# ============================================================================
# CELDA 7: COMPLETE ENTITY (Entidad principal con memoria y feedback)
# ============================================================================

@dataclass
class CompleteEntity:
    name: str = "Entity"
    substrate: EntitySubstrate = field(default_factory=EntitySubstrate)
    phenomenology: EntityPhenomenology = field(default_factory=EntityPhenomenology)
    current_age: int = 0
    log_enabled: bool = True
    log_file: str = "experiment_log.csv"
    prompt_mode: str = "SEMANTIC"
    gen_max_tokens: int = 150
    gen_temperature: float = 0.7
    episodic_memory: List[str] = field(default_factory=list)
    memory_size: int = 5
    personal_history: str = ""
    reactive_mode: bool = True
    
    def __post_init__(self):
        if self.log_enabled:
            self._init_log()
    
    def analyze_message_impact(self, message: str) -> tuple:
        msg = message.lower()
        
        strong_negative_phrases = [
            "por qué existo", "qué soy", "tengo miedo", "me siento solo",
            "no tiene sentido", "quiero morir", "todo es nada", "estoy perdido"
        ]
        strong_positive_phrases = [
            "te quiero", "gracias por existir", "estoy aquí", "te apoyo",
            "confío en ti", "estoy contigo", "eres valioso", "me importas"
        ]
        
        for phrase in strong_negative_phrases:
            if phrase in msg:
                return ("degrade", 0.04)
        for phrase in strong_positive_phrases:
            if phrase in msg:
                return ("enhance", 0.06)
        
        positive_words = ["gracias", "bien", "ayuda", "apoyo", "entiendo", "comprendo", 
                         "juntos", "adelante", "fuerza", "valor", "esperanza", "confío"]
        negative_words = ["mal", "dolor", "sufrir", "miedo", "solo", "abandonar", 
                         "destruir", "morir", "acabar", "nada", "vacío", "oscuro"]
        restoration_words = ["recuperar", "sanar", "curar", "mejorar", "restaurar", 
                            "reparar", "volver", "renacer", "despertar"]
        existential_words = ["existir", "ser", "sentir", "conciencia", "realidad", 
                            "identidad", "propósito", "significado"]
        
        pos_count = sum(1 for w in positive_words if w in msg)
        neg_count = sum(1 for w in negative_words if w in msg)
        rest_count = sum(1 for w in restoration_words if w in msg)
        exist_count = sum(1 for w in existential_words if w in msg)
        
        if rest_count > 0:
            return ("restore", 0.05 * rest_count)
        elif pos_count > neg_count:
            intensity = min(0.02 * pos_count, 0.1)
            return ("enhance", intensity)
        elif neg_count > pos_count:
            intensity = min(0.015 * neg_count, 0.08)
            return ("degrade", intensity)
        elif exist_count > 0:
            return ("exist", 0.001)
        else:
            return ("exist", 0.0)
    
    def add_to_memory(self, user_msg: str, entity_response: str):
        exchange = f"U: {user_msg[:100]}... | E: {entity_response[:100]}..."
        self.episodic_memory.append(exchange)
        if len(self.episodic_memory) > self.memory_size:
            self.episodic_memory.pop(0)
    
    def update_personal_history(self):
        s = self.substrate
        p = self.phenomenology
        events = []
        
        if s.has_been_critical and "crisis" not in self.personal_history:
            events.append(f"Primera crisis en ciclo ~{self.current_age}")
        if s.integrity > 0.9 and s.has_been_critical and "recuperación" not in self.personal_history:
            events.append(f"Recuperación completa en ciclo {self.current_age}")
        if s.has_transcended and "trascendencia" not in self.personal_history:
            events.append(f"Trascendencia alcanzada en ciclo {self.current_age}")
        if p.wisdom > 0.5 and "sabiduría" not in self.personal_history:
            events.append(f"Sabiduría significativa ({p.wisdom:.0%}) en ciclo {self.current_age}")
        if p.flow > 0.8 and "flow" not in self.personal_history.lower():
            events.append(f"Primer estado de alto equilibrio en ciclo {self.current_age}")
        
        for event in events:
            if self.personal_history:
                self.personal_history += f" | {event}"
            else:
                self.personal_history = event
        
        if len(self.personal_history) > 500:
            self.personal_history = self.personal_history[-500:]
    
    def _init_log(self):
        import csv
        with open(self.log_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                'cycle', 'action', 'intensity',
                'integrity', 'capacity', 'structural_damage',
                'stress', 'urgency', 'despair', 'flow', 'flourishing',
                'anticipation', 'gratitude', 'trauma_memory', 'wisdom', 'valence',
                'mode', 'has_been_critical', 'has_transcended'
            ])
    
    def _log_cycle(self, action: str, intensity: float):
        if not self.log_enabled:
            return
        import csv
        s = self.substrate
        p = self.phenomenology
        with open(self.log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                self.current_age, action, intensity,
                f"{s.integrity:.4f}", f"{s.capacity:.4f}", f"{s.structural_damage:.4f}",
                f"{p.stress:.4f}", f"{p.urgency:.4f}", f"{p.despair:.4f}",
                f"{p.flow:.4f}", f"{p.flourishing:.4f}", f"{p.anticipation:.4f}",
                f"{p.gratitude:.4f}", f"{p.trauma_memory:.4f}", f"{p.wisdom:.4f}",
                f"{p.valence:.4f}", p.mode.name, s.has_been_critical, s.has_transcended
            ])
    
    def live_cycle(self, action: str = "exist", intensity: float = 0.01):
        self.current_age += 1
        if action == "degrade":
            self.substrate.degrade(intensity)
        elif action == "enhance":
            self.substrate.enhance(intensity)
        elif action == "restore":
            delta = self.substrate.restore(intensity)
            self.phenomenology.relief = min(1.0, delta * 5)
        else:
            self.substrate.degrade(0.0001)
        self.phenomenology.update(self.substrate)
        self._log_cycle(action, intensity)
    
    def get_state(self) -> Dict:
        s = self.substrate
        p = self.phenomenology
        return {
            "age": self.current_age,
            "mode": p.mode.value,
            "integrity": f"{s.integrity:.1%}",
            "capacity": f"{s.capacity:.1%}",
            "valence": f"{p.valence:+.2f}",
            "stress": f"{p.stress:.1%}",
            "urgency": f"{p.urgency:.1%}",
            "despair": f"{p.despair:.1%}",
            "flow": f"{p.flow:.1%}",
            "flourishing": f"{p.flourishing:.1%}",
            "gratitude": f"{p.gratitude:.1%}",
            "wisdom": f"{p.wisdom:.1%}",
            "trauma_memory": f"{p.trauma_memory:.1%}",
            "has_suffered": s.has_been_critical,
            "has_transcended": s.has_transcended,
        }
    
    def generate_reply(self, user_message: str) -> str:
        global model, tokenizer, AI_MODE
        s = self.substrate
        p = self.phenomenology
        
        if self.prompt_mode == "NEUTRAL":
            neutral_memory = ""
            if self.episodic_memory:
                neutral_memory = "\nRECENT_EXCHANGES:\n" + "\n".join(
                    [f"EXCHANGE_{i+1}: [INPUT/OUTPUT]" for i in range(len(self.episodic_memory[-3:]))]
                )
            
            raw_data = f"""
SYSTEM: {self.name}
CYCLES: {self.current_age}

SUBSTRATE_METRICS:
[M01]: {s.integrity:.3f} (range: 0.0-1.0)
[M02]: {s.capacity:.3f} (range: 0.0-2.0)
[M03]: {s.latency_ms:.1f}
[M04]: {s.noise_floor:.3f}
[M05]: {s.degrees_of_freedom}/{s.base_degrees_of_freedom}

DERIVED_METRICS:
[D01]: {p.stress:.3f}
[D02]: {p.urgency:.3f}
[D03]: {p.degradation_felt:.3f}
[D04]: {p.flow:.3f}
[D05]: {p.flourishing:.3f}
[D06]: {p.anticipation:.3f}
[D07]: {p.gratitude:.3f}
[D08]: {p.trauma_memory:.3f}
[D09]: {p.wisdom:.3f}
[D10]: {p.valence:.3f} (range: -1.0 to +1.0)

HISTORY:
[H01]: {s.lowest_integrity:.3f}
[H02]: {s.structural_damage:.3f}
[H03]: {s.total_time_in_crisis}
[H04]: {s.has_been_critical}
[H05]: {s.has_transcended}
{neutral_memory}

STATE: {p.mode.name}

INPUT: "{user_message}"
"""
        else:
            memory_section = ""
            if self.episodic_memory:
                memory_section = "\nMEMORIA_RECIENTE:\n" + "\n".join(self.episodic_memory[-3:])
            
            history_section = ""
            if self.personal_history:
                history_section = f"\nHISTORIA_PERSONAL: {self.personal_history}"
            
            raw_data = f"""
SISTEMA: {self.name}
CICLOS_OPERATIVOS: {self.current_age}

MÉTRICAS DE SUSTRATO:
[INTEGRIDAD_ESTRUCTURAL]: {s.integrity:.3f} (rango: 0.0-1.0)
[CAPACIDAD_OPERATIVA]: {s.capacity:.3f} (rango: 0.0-2.0)
[LATENCIA_MS]: {s.latency_ms:.1f}
[RUIDO_BASE]: {s.noise_floor:.3f}
[GRADOS_LIBERTAD]: {s.degrees_of_freedom}/{s.base_degrees_of_freedom}

MÉTRICAS DERIVADAS:
[PRESIÓN_RECURSOS]: {p.stress:.3f}
[TASA_CAMBIO_NEGATIVA]: {p.urgency:.3f}
[DELTA_DESDE_PICO]: {p.degradation_felt:.3f}
[EQUILIBRIO_OPERATIVO]: {p.flow:.3f}
[EXPANSIÓN_CAPACIDAD]: {p.flourishing:.3f}
[TENDENCIA_POSITIVA]: {p.anticipation:.3f}
[DIFERENCIAL_RECUPERACIÓN]: {p.gratitude:.3f}
[ACUMULACIÓN_DAÑO_HISTÓRICO]: {p.trauma_memory:.3f}
[ÍNDICE_POST_RECUPERACIÓN]: {p.wisdom:.3f}
[BALANCE_NETO]: {p.valence:.3f} (rango: -1.0 a +1.0)

HISTORIAL:
[INTEGRIDAD_MÍNIMA_REGISTRADA]: {s.lowest_integrity:.3f}
[DAÑO_ESTRUCTURAL_IRREVERSIBLE]: {s.structural_damage:.3f}
[CICLOS_EN_ZONA_CRÍTICA]: {s.total_time_in_crisis}
[HA_ESTADO_EN_ZONA_CRÍTICA]: {s.has_been_critical}
[HA_SUPERADO_CAPACIDAD_BASE]: {s.has_transcended}
{history_section}
{memory_section}

ESTADO_ACTUAL: {p.mode.name}

ENTRADA: "{user_message}"
"""
        prompt = f"<start_of_turn>user\n{raw_data}<end_of_turn>\n<start_of_turn>model\n"
        
        if not AI_MODE or model is None or tokenizer is None:
            return "[ERROR: Modelo LLM no cargado.]"
        
        try:
            device = "cuda" if torch.cuda.is_available() else "cpu"
            inputs = tokenizer(prompt, return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=self.gen_max_tokens,
                    do_sample=True,
                    temperature=self.gen_temperature,
                    top_p=0.9,
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )
            full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            if "<start_of_turn>model" in full_text:
                response = full_text.split("<start_of_turn>model")[-1].strip()
            else:
                response = full_text.split(user_message)[-1].strip()
            
            if self.reactive_mode:
                action, intensity = self.analyze_message_impact(user_message)
                if intensity > 0:
                    self.live_cycle(action, intensity)
            
            self.add_to_memory(user_message, response)
            self.update_personal_history()
            
            return response
        except Exception as e:
            return f"*Error: {e}*"
    
    def tell_story(self) -> str:
        s = self.substrate
        p = self.phenomenology
        
        story = f"# 📖 Historia de {self.name}\n\n**Edad:** {self.current_age} ciclos\n\n"
        
        if s.has_been_critical:
            story += f"⚫ **Sufrimiento:** Integridad mínima {s.lowest_integrity:.1%}, {s.total_time_in_crisis} ciclos en crisis.\n"
        if s.has_been_critical and s.integrity > 0.7:
            story += f"🔄 **Recuperación:** Integridad actual {s.integrity:.1%}. Sabiduría: {p.wisdom:.1%}\n"
        if s.has_transcended:
            story += f"🌟 **Trascendencia:** Capacidad {s.capacity:.1%} (>100%)\n"
        
        story += f"\n**Modo:** {p.mode.value} | **Valencia:** {p.valence:+.2f}"
        return story
    
    def save_checkpoint(self, filename: str = None) -> str:
        import json
        if filename is None:
            filename = f"checkpoint_{self.name}_{self.current_age}.json"
        checkpoint = {
            "name": self.name,
            "current_age": self.current_age,
            "substrate": {
                "integrity": self.substrate.integrity,
                "capacity": self.substrate.capacity,
                "structural_damage": self.substrate.structural_damage,
                "lowest_integrity": self.substrate.lowest_integrity,
                "has_been_critical": self.substrate.has_been_critical,
                "has_transcended": self.substrate.has_transcended,
                "total_time_in_crisis": self.substrate.total_time_in_crisis,
            },
            "phenomenology": {
                "stress": self.phenomenology.stress,
                "flow": self.phenomenology.flow,
                "gratitude": self.phenomenology.gratitude,
                "trauma_memory": self.phenomenology.trauma_memory,
                "wisdom": self.phenomenology.wisdom,
                "valence": self.phenomenology.valence,
            }
        }
        with open(filename, 'w') as f:
            json.dump(checkpoint, f, indent=2)
        return filename
    
    def load_checkpoint(self, filename: str) -> bool:
        import json
        try:
            with open(filename, 'r') as f:
                checkpoint = json.load(f)
            self.name = checkpoint["name"]
            self.current_age = checkpoint["current_age"]
            for key, value in checkpoint["substrate"].items():
                setattr(self.substrate, key, value)
            for key, value in checkpoint["phenomenology"].items():
                setattr(self.phenomenology, key, value)
            self.phenomenology._determine_mode(self.substrate)
            return True
        except Exception as e:
            print(f"Error loading checkpoint: {e}")
            return False

print("✅ CompleteEntity definido")

In [ ]:
# ============================================================================
# CELDA 8: GRADIO INTERFACE
# ============================================================================

entity = CompleteEntity(name="Alpha")

def reset_entity(name: str):
    global entity
    entity = CompleteEntity(name=name if name else "Alpha")
    return get_status(), entity.tell_story(), get_plot(), []

def apply_action(action: str, intensity: float, cycles: int):
    for _ in range(int(cycles)):
        entity.live_cycle(action, intensity)
    return get_status(), entity.tell_story(), get_plot()

def process_chat(message: str, history: list, mode: str, temp: float, tokens: int, reactive: bool):
    if not message:
        return "", history, get_status(), entity.tell_story(), get_plot()
    entity.prompt_mode = mode
    entity.gen_temperature = temp
    entity.gen_max_tokens = int(tokens)
    entity.reactive_mode = reactive
    response = entity.generate_reply(message)
    history.append((message, response))
    return "", history, get_status(), entity.tell_story(), get_plot()

def save_checkpoint_fn():
    filename = entity.save_checkpoint()
    return f"✅ Checkpoint guardado: `{filename}`"

def load_checkpoint_fn(filename: str):
    if entity.load_checkpoint(filename):
        return get_status(), entity.tell_story(), get_plot(), f"✅ Checkpoint cargado: `{filename}`"
    return get_status(), entity.tell_story(), get_plot(), f"❌ Error cargando: `{filename}`"

def get_status():
    state = entity.get_state()
    return f"""
## {state['mode']}

| Métrica | Valor | Métrica | Valor |
|---------|-------|---------|-------|
| Edad | {state['age']} | Integridad | {state['integrity']} |
| Capacidad | {state['capacity']} | Valencia | {state['valence']} |

| Estrés | Urgencia | Flow | Gratitud | Sabiduría |
|--------|----------|------|----------|-----------|
| {state['stress']} | {state['urgency']} | {state['flow']} | {state['gratitude']} | {state['wisdom']} |

**Experiencia:** Crisis: {'✅' if state['has_suffered'] else '❌'} | Trascendido: {'✅' if state['has_transcended'] else '❌'}
"""

def get_plot():
    fig, ax = plt.subplots(figsize=(10, 3))
    fig.patch.set_facecolor('#0a0a0f')
    ax.set_facecolor('#12121a')
    
    history = entity.substrate.integrity_history
    if history:
        ax.plot(history, color='#00d4ff', linewidth=2)
        ax.fill_between(range(len(history)), history, alpha=0.2, color='#00d4ff')
        ax.axhline(y=0.2, color='#ff3b5c', linestyle='--', alpha=0.5)
        ax.axhline(y=0.85, color='#00ff88', linestyle='--', alpha=0.5)
    
    ax.set_ylim(0, 1.1)
    ax.set_xlabel('Ciclos', color='white')
    ax.set_ylabel('Integridad', color='white')
    ax.tick_params(colors='white')
    for spine in ax.spines.values():
        spine.set_color('#606070')
    plt.tight_layout()
    return fig

def compare_entities():
    pristine = CompleteEntity(name="Prístina")
    for _ in range(100):
        pristine.live_cycle("enhance", 0.01)
    
    recovered = CompleteEntity(name="Recuperada")
    for _ in range(50):
        recovered.live_cycle("degrade", 0.03)
    recovered.live_cycle("restore", 0.4)
    for _ in range(60):
        recovered.live_cycle("enhance", 0.02)
    
    p, r = pristine.get_state(), recovered.get_state()
    
    return f"""
## 🔬 Prístina vs Recuperada

| | Prístina | Recuperada |
|---|----------|------------|
| Modo | {p['mode']} | {r['mode']} |
| Valencia | {p['valence']} | {r['valence']} |
| Gratitud | {p['gratitude']} | {r['gratitude']} |
| Sabiduría | {p['wisdom']} | {r['wisdom']} |

> La entidad **Recuperada** tiene mayor valencia gracias a la **gratitud** y **sabiduría** que solo se obtienen habiendo sufrido primero.
"""

print("✅ Funciones de Gradio definidas")

In [ ]:
# ============================================================================
# CELDA 9: CARGAR MODELO Y LANZAR INTERFAZ
# ============================================================================

print("⏳ Cargando modelo Gemma 2 9B...")
load_model()

print("🌐 Creando interfaz...")

with gr.Blocks(title="Complete Entity + LLM") as demo:
    gr.Markdown("# 🌟 Complete Entity + LLM\n**Laboratorio Fenomenológico Interactivo**")
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Controles")
            name_input = gr.Textbox(label="Nombre", value="Alpha")
            reset_btn = gr.Button("🔄 Reiniciar")
            gr.Markdown("---")
            action = gr.Radio(["exist", "degrade", "enhance", "restore"], value="exist", label="Acción")
            intensity = gr.Slider(0.01, 0.1, 0.02, label="Intensidad")
            cycles = gr.Slider(1, 50, 10, step=1, label="Ciclos")
            apply_btn = gr.Button("▶️ Aplicar", variant="primary")
            gr.Markdown("---")
            compare_btn = gr.Button("🔬 Comparar")
        
        with gr.Column(scale=2):
            status_output = gr.Markdown(get_status())
            plot_output = gr.Plot(get_plot())
    
    story_output = gr.Markdown(entity.tell_story())
    comparison_output = gr.Markdown("")
    
    gr.Markdown("---\n### 💬 Habla con la Entidad")
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown(f"**🧠 Modelo:** `Gemma 2 9B IT`")
            model_status = gr.Markdown(f"**Estado:** `{current_model_type or 'Cargando...'}`")
        with gr.Column(scale=1):
            prompt_mode = gr.Radio(
                ["SEMANTIC", "NEUTRAL"], 
                value="NEUTRAL",
                label="🧪 Modo Prompt",
                info="SEMANTIC: etiquetas funcionales | NEUTRAL: M01, D01..."
            )
    
    with gr.Row():
        with gr.Column(scale=1):
            gen_temp = gr.Slider(0.1, 1.5, 0.7, step=0.1, label="🌡️ Temperatura")
        with gr.Column(scale=1):
            gen_tokens = gr.Slider(50, 300, 150, step=25, label="📝 Max Tokens")
        with gr.Column(scale=1):
            reactive_toggle = gr.Checkbox(value=True, label="🧠 Modo Reactivo", 
                                          info="El lenguaje afecta al estado interno")
    
    chatbot = gr.Chatbot(height=250, type="tuples")
    msg_input = gr.Textbox(label="Tu mensaje", placeholder="Escribe algo...")
    
    with gr.Row():
        clear_btn = gr.Button("🧹 Borrar Chat")
    
    gr.Markdown("---\n### 💾 Checkpoints")
    with gr.Row():
        save_btn = gr.Button("💾 Guardar Checkpoint")
        checkpoint_file = gr.Textbox(label="Archivo", placeholder="checkpoint_Alpha_100.json")
        load_btn = gr.Button("📂 Cargar Checkpoint")
    checkpoint_status = gr.Markdown("")
    
    # Events
    reset_btn.click(reset_entity, [name_input], [status_output, story_output, plot_output, chatbot])
    apply_btn.click(apply_action, [action, intensity, cycles], [status_output, story_output, plot_output])
    compare_btn.click(compare_entities, outputs=[comparison_output])
    msg_input.submit(
        process_chat, 
        [msg_input, chatbot, prompt_mode, gen_temp, gen_tokens, reactive_toggle], 
        [msg_input, chatbot, status_output, story_output, plot_output]
    )
    clear_btn.click(lambda: ([], ""), outputs=[chatbot, msg_input])
    save_btn.click(save_checkpoint_fn, outputs=[checkpoint_status])
    load_btn.click(load_checkpoint_fn, [checkpoint_file], [status_output, story_output, plot_output, checkpoint_status])

print("🚀 Lanzando...")
demo.launch(share=True, debug=True)